In [3]:
from google.colab import drive
drive.mount('/content/drive')


Mounted at /content/drive


In [4]:
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.models import Sequential
from keras.layers import Dense, Embedding, LSTM, SpatialDropout1D
from matplotlib import pyplot
from sklearn.model_selection import train_test_split
from keras.utils import to_categorical

import re

from sklearn.preprocessing import LabelEncoder

data = pd.read_csv('/content/drive/MyDrive/Sentiment.csv')
# Keeping only the neccessary columns
data = data[['text','sentiment']]

data['text'] = data['text'].apply(lambda x: x.lower())
data['text'] = data['text'].apply((lambda x: re.sub('[^a-zA-z0-9\s]', '', x)))

for idx, row in data.iterrows():
    row[0] = row[0].replace('rt', ' ')

max_fatures = 2000
tokenizer = Tokenizer(num_words=max_fatures, split=' ')
tokenizer.fit_on_texts(data['text'].values)
X = tokenizer.texts_to_sequences(data['text'].values)

X = pad_sequences(X)

embed_dim = 128
lstm_out = 196
def createmodel():
    model = Sequential()
    model.add(Embedding(max_fatures, embed_dim,input_length = X.shape[1]))
    model.add(LSTM(lstm_out, dropout=0.2, recurrent_dropout=0.2))
    model.add(Dense(3,activation='softmax'))
    model.compile(loss = 'categorical_crossentropy', optimizer='adam',metrics = ['accuracy'])
    return model
# print(model.summary())

labelencoder = LabelEncoder()
integer_encoded = labelencoder.fit_transform(data['sentiment'])
y = to_categorical(integer_encoded)
X_train, X_test, Y_train, Y_test = train_test_split(X,y, test_size = 0.33, random_state = 42)

batch_size = 32
model = createmodel()
model.fit(X_train, Y_train, epochs = 1, batch_size=batch_size, verbose = 2)
score,acc = model.evaluate(X_test,Y_test,verbose=2,batch_size=batch_size)
print(score)
print(acc)
print(model.metrics_names)


291/291 - 53s - loss: 0.8242 - accuracy: 0.6428 - 53s/epoch - 181ms/step
144/144 - 2s - loss: 0.7569 - accuracy: 0.6645 - 2s/epoch - 11ms/step
0.7568740844726562
0.6644822955131531
['loss', 'accuracy']


In [5]:
!pip install --upgrade tensorflow



In [7]:
!pip install scikeras



In [8]:
  import pandas as pd
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.models import Sequential, load_model
from keras.layers import Dense, Embedding, LSTM
from keras.utils import to_categorical
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.preprocessing import LabelEncoder
import re
from scikeras.wrappers import KerasClassifier

# Load the data
data = pd.read_csv('/content/drive/MyDrive/Sentiment.csv')
data = data[['text','sentiment']]

# Preprocess the text
data['text'] = data['text'].apply(lambda x: x.lower())
data['text'] = data['text'].apply((lambda x: re.sub('[^a-zA-z0-9\s]', '', x)))

for idx, row in data.iterrows():
    row[0] = row[0].replace('rt', ' ')

# Tokenization
max_fatures = 2000
tokenizer = Tokenizer(num_words=max_fatures, split=' ')
tokenizer.fit_on_texts(data['text'].values)
X = tokenizer.texts_to_sequences(data['text'].values)
X = pad_sequences(X)

# Model definition
embed_dim = 128
lstm_out = 196

def createmodel(optimizer='adam'):
    model = Sequential()
    model.add(Embedding(max_fatures, embed_dim, input_length=X.shape[1]))
    model.add(LSTM(lstm_out, dropout=0.2, recurrent_dropout=0.2))
    model.add(Dense(3, activation='softmax'))
    model.compile(loss='categorical_crossentropy', optimizer=optimizer, metrics=['accuracy'])
    return model

# Prepare the dataset
labelencoder = LabelEncoder()
integer_encoded = labelencoder.fit_transform(data['sentiment'])
y = to_categorical(integer_encoded)
X_train, X_test, Y_train, Y_test = train_test_split(X, y, test_size=0.33, random_state=42)

# Train the model
batch_size = 32
model = createmodel()
model.fit(X_train, Y_train, epochs=1, batch_size=batch_size, verbose=2)
score, acc = model.evaluate(X_test, Y_test, verbose=2, batch_size=batch_size)
print(score)
print(acc)

# Save the model
model.save('lstm_sentiment_model.keras')

# Load the model
model = load_model('lstm_sentiment_model.keras')

# Function to preprocess new text data
def preprocess_text(text):
    text = text.lower()
    text = re.sub('[^a-zA-z0-9\s]', '', text)
    text = text.replace('rt', ' ')
    sequence = tokenizer.texts_to_sequences([text])
    padded = pad_sequences(sequence, maxlen=X.shape[1])
    return padded

# Predict on new text data
new_text = "A lot of good things are happening. We are respected again throughout the world, and that's a great thing.@realDonaldTrump"
new_text_processed = preprocess_text(new_text)
prediction = model.predict(new_text_processed)
predicted_class = labelencoder.inverse_transform([prediction.argmax(axis=-1)])
print(f"Predicted sentiment: {predicted_class[0]}")

# Apply GridSearchCV on the source code
def create_model_for_gridsearch(optimizer='adam'):
    return createmodel(optimizer=optimizer)

# Wrap the model using the KerasClassifier
model = KerasClassifier(model=create_model_for_gridsearch, epochs=1, batch_size=batch_size)

# Define the grid search parameters
param_grid = {
    'optimizer': ['adam', 'rmsprop'],
    'epochs': [1, 3],
    'batch_size': [32, 64]
}

# Perform the grid search
grid = GridSearchCV(estimator=model, param_grid=param_grid, cv=3)
grid_result = grid.fit(X_train, Y_train)

# Print the results
print("Best: %f using %s" % (grid_result.best_score_, grid_result.best_params_))


291/291 - 57s - loss: 0.8278 - accuracy: 0.6412 - 57s/epoch - 195ms/step
144/144 - 2s - loss: 0.7730 - accuracy: 0.6682 - 2s/epoch - 11ms/step


0.7729532718658447
0.6681957244873047
1/1 [==============================] - 0s 359ms/step


/usr/local/lib/python3.10/dist-packages/sklearn/preprocessing/_label.py:155: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


Predicted sentiment: Negative
97/97 [==============================] - 1s 9ms/step


97/97 [==============================] - 1s 9ms/step


...

Epoch 1/3
97/97 [==============================] - 24s 198ms/step - loss: 0.8702 - accuracy: 0.6210
Epoch 2/3
97/97 [==============================] - 15s 158ms/step - loss: 0.6960 - accuracy: 0.7009
Epoch 3/3
49/49 [==============================] - 1s 9ms/step


Epoch 1/3
97/97 [==============================] - 21s 185ms/step - loss: 0.8707 - accuracy: 0.6228
Epoch 2/3
97/97 [==============================] - 16s 165ms/step - loss: 0.7233 - accuracy: 0.6873
Epoch 3/3
49/49 [==============================] - 1s 9ms/step


Epoch 1/3
97/97 [==============================] - 24s 192ms/step - loss: 0.8757 - accuracy: 0.6215
Epoch 2/3
97/97 [==============================] - 15s 154ms/step - loss: 0.7072 - accuracy: 0.6965
Epoch 3/3
49/49 [==============================] - 1s 13ms/step


Epoch 1/3
97/97 [==============================] - 24s 202ms/step - loss: 0.8836 - accuracy: 0.6164
Epoch 2/3
97/97 [==============================] - 15s 150ms/step - loss: 0.7040 - accuracy: 0.6985
Epoch 3/3
49/49 [==============================] - 1s 9ms/step


291/291 [==============================] - 50s 159ms/step - loss: 0.8205 - accuracy: 0.6466
Best: 0.671044 using {'batch_size': 32, 'epochs': 1, 'optimizer': 'rmsprop'}
